<a href="https://colab.research.google.com/github/diqnfl777/2022F-Ajou-ML-TEAM3/blob/ldhbranch/ml_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT finetuning

In [30]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-klb0ra81
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-klb0ra81


In [31]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [32]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [33]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [34]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [35]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# KOR PARAMETERS

In [36]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

frac = 1
test_size = 0.2
dr_rate = 0.7

# 데이터 불러오기

In [37]:
# !wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
# !wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

In [38]:
# dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
# dataset_test = nlp.data.TSVDataset(".cache/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
#데이터 가공
#청구기호 숫자 앞 2개만 따와서 각각 매핑. 65~70은 예외적으로 경영학
import pandas as pd
import re 
data = pd.read_csv("/content/gdrive/MyDrive/LibraryCsv/hapbon.csv", encoding = "cp949")
data = data.loc[:,['제목', '청구번호']]
data['청구번호'] = data['청구번호'].apply(str)

data['청구번호'] = data['청구번호'].replace({r'(.*?)(\d{2})\d.*' : r'\2'}, regex=True)

data['청구번호'] = pd.to_numeric(data['청구번호'])
data.loc[(data['청구번호'] < 10), ['청구번호']] = 1000
data.loc[(data['청구번호'] < 20), ['청구번호']] = 1001
data.loc[(data['청구번호'] < 30), ['청구번호']] = 1002
data.loc[(data['청구번호'] < 40), ['청구번호']] = 1003
data.loc[(data['청구번호'] < 50), ['청구번호']] = 1004
data.loc[(data['청구번호'] < 60), ['청구번호']] = 1005
data.loc[(data['청구번호'] < 65), ['청구번호']] = 1006 #경영학쪽은 따로 분류이기 떄문에 65 사용
data.loc[(data['청구번호'] < 70), ['청구번호']] = 1007 #즉, 1007 쪽이 경영학책
data.loc[(data['청구번호'] < 80), ['청구번호']] = 1008
data.loc[(data['청구번호'] < 90), ['청구번호']] = 1009
data.loc[(data['청구번호'] <  100), ['청구번호']] = 1010

labels = {'0':'총류',
          '1':'철학',
          '2':'종교',
          '3':'사회학',
          '4':'언어',
          '5':'자연과학',
          '6':'기술과학',
          '7':'경영학',
          '8':'예술',
          '9':'문학',
          '10':'역사'
          }

data['청구번호'] = data['청구번호']%1000
data['청구번호'] = data['청구번호'].astype(int)
# data['청구번호'] = data['청구번호'].apply(str)
# data['청구번호'] = data['청구번호'].replace(labels)

#input_string = "Peace-building and development in Guatemala and Northern Ireland"
regex = '.*[ㄱ-ㅎ|ㅏ-ㅣ|가-힣].*'
kor_data = data[data.제목.str.match(regex)]                                 # 한글이 있다면 따로 빼내기
eng_data = pd.concat([data, kor_data, kor_data]).drop_duplicates(keep=False)

kor_data

,제목,청구번호
1,대학수학능력시험의 영어 독해문제 분석 및 개선방안,0
2,韓民族 統一聯邦國家 建設을 위한 올바른 歷史의 認識과 그 成就를 위한 提言 : 지...,0
5,위험한 생각들,0
7,백만장자 메신저 :당신의 경험이 돈이 되는 순간이 온다,0
8,지식 : 그 탄생과 유통에 대한 모든 지식,0
...,...,...
436309,콜롬비아 便覽,10
436310,페루 便覽,10
436311,파라과이 便覽,10
436312,우루과이 便覽,10


In [41]:
kor_data = kor_data.sample(frac = frac, random_state = 1)
# kor_data.to_csv("/content/gdrive/MyDrive/LibraryCsv/hapbonkordata.txt", sep="\t")
from sklearn.model_selection import train_test_split

train, test = train_test_split(kor_data, test_size=test_size)
train.to_csv("/content/gdrive/MyDrive/LibraryCsv/train.txt", sep="\t")
test.to_csv("/content/gdrive/MyDrive/LibraryCsv/test.txt", sep="\t")

In [42]:
dataset_train = nlp.data.TSVDataset("/content/gdrive/MyDrive/LibraryCsv/train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/gdrive/MyDrive/LibraryCsv/test.txt", field_indices=[1,2], num_discard_samples=1)

# KOR

In [43]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [44]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [45]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [46]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [47]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=11,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [48]:
model = BERTClassifier(bertmodel, dr_rate=dr_rate).to(device)

In [49]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [50]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [51]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [52]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [53]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [54]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/2919 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.394139289855957 train acc 0.078125
epoch 1 batch id 201 loss 2.1951069831848145 train acc 0.20366915422885573
epoch 1 batch id 401 loss 1.6241918802261353 train acc 0.2943812344139651
epoch 1 batch id 601 loss 1.3018819093704224 train acc 0.37856177204658903
epoch 1 batch id 801 loss 1.336676001548767 train acc 0.4428643882646692
epoch 1 batch id 1001 loss 1.065588116645813 train acc 0.4878090659340659
epoch 1 batch id 1201 loss 0.9210886359214783 train acc 0.5180578684429642
epoch 1 batch id 1401 loss 1.1655889749526978 train acc 0.5416109029264811
epoch 1 batch id 1601 loss 0.9938894510269165 train acc 0.5608408806995627
epoch 1 batch id 1801 loss 0.7581765055656433 train acc 0.5758259300388673
epoch 1 batch id 2001 loss 0.81058669090271 train acc 0.5890492253873063
epoch 1 batch id 2201 loss 0.8270381689071655 train acc 0.5993227510222626
epoch 1 batch id 2401 loss 1.00700044631958 train acc 0.6087047063723449
epoch 1 batch id 2601 loss 0.8042713403701782 t

  0%|          | 0/730 [00:00<?, ?it/s]

epoch 1 test acc 0.7309246575342466


  0%|          | 0/2919 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7588099315068493


  0%|          | 0/2919 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4161885976791382 train acc 0.890625
epoch 5 batch id 201 loss 0.37124496698379517 train acc 0.8760883084577115
epoch 5 batch id 401 loss 0.5260561108589172 train acc 0.8744934538653366
epoch 5 batch id 601 loss 0.21782787144184113 train acc 0.873830074875208
epoch 5 batch id 801 loss 0.33348456025123596 train acc 0.8747464107365793
epoch 5 batch id 1001 loss 0.20960569381713867 train acc 0.8772321428571429
epoch 5 batch id 1201 loss 0.3084869682788849 train acc 0.8780573480432973
epoch 5 batch id 1401 loss 0.37942105531692505 train acc 0.8798625981441828
epoch 5 batch id 1601 loss 0.4174521565437317 train acc 0.881821908182386
epoch 5 batch id 1801 loss 0.2802095413208008 train acc 0.8834241393670184
epoch 5 batch id 2001 loss 0.1511903703212738 train acc 0.8847763618190905
epoch 5 batch id 2201 loss 0.3105580806732178 train acc 0.8854001022262608
epoch 5 batch id 2401 loss 0.37809592485427856 train acc 0.8857246980424823
epoch 5 batch id 2601 loss 0.263087689

  0%|          | 0/730 [00:00<?, ?it/s]

epoch 5 test acc 0.7616780821917808


# SAVE

In [55]:
def save_checkpoint(save_path, model, valid_loss):# ------ 모델 평가를 위해 훈련 과정을 저장
    if save_path == None:
        return
    state_dict = {'model_state_dict': model.state_dict(), 'valid_loss': valid_loss}
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):# ------ save_checkpoint 함수에서 저장된 모델을 가져옵니다.
    if load_path == None:
        return
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [56]:
save_checkpoint("/content/gdrive/MyDrive/learned.multiLangModel", model, 0.325)

Model saved to ==> /content/gdrive/MyDrive/learned.multiLangModel


# TEST

In [149]:
from pandas import DataFrame

toselftest = ["양자역학 이야기"]

labels = {0:'총류',
          1:'철학',
          2:'종교',
          3:'사회학',
          4:'언어',
          5:'자연과학',
          6:'기술과학',
          7:'경영학',
          8:'예술',
          9: '문학',
          10:'역사'
}

selftest = DataFrame({
  'value': toselftest,
  'null': [0 for x in toselftest]
})

selftest.to_csv("/content/gdrive/MyDrive/LibraryCsv/selftest.txt", sep="\t")
dataset_selftest = nlp.data.TSVDataset("/content/gdrive/MyDrive/LibraryCsv/selftest.txt", field_indices=[1,2], num_discard_samples=1)
data_selftest = BERTDataset(dataset_selftest, 0, 1, tok, max_len, True, False)
selftest_dataloader = torch.utils.data.DataLoader(data_selftest, batch_size=1, num_workers=1)

for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(selftest_dataloader), total=len(selftest_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)

      prediction = out.cpu().detach().numpy()[0]
      minval = prediction.min()
      prediction = [x-minval for x in prediction]
      sumval = sum(prediction)
      print(toselftest[batch_id])
      for index, score in sorted(enumerate(prediction), reverse = True, key = lambda prediction:prediction[1]):
        print("{} {}%".format(labels[index], score*100/sumval))


  0%|          | 0/1 [00:00<?, ?it/s]

양자역학 이야기
자연과학 52.66125925081528%
기술과학 19.053975993516787%
경영학 7.34884896355313%
사회학 5.102068721764783%
총류 3.6715220460003852%
예술 3.3982341068802775%
종교 2.872254048246956%
철학 2.5458705113216547%
언어 2.0182711642034725%
역사 1.3276951936972805%
문학 0.0%
